In [2]:
import pandas as pd
import numpy as np
import requests
from IPython.display import display_html
from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes  !conda install -c conda-forge folium=0.5.0 --yes
import folium


In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
# Importing Geospatial data
df = pd.read_csv('http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
# downloading Wikipedia data of postal code to merge above and this data 

net = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(net,'lxml')
tab = str(soup.table)
display_html(tab)

In [5]:
htm = pd.read_html(tab)
df2 = htm[0]
df2.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df2.head()
df2 = df2[df2['Borough']!='Not assigned']


In [7]:
df2_1 = df2.groupby(['Postal Code'],sort=False).agg(','.join)
df2_1.reset_index(inplace=True)
df2_1.head()
df2_1['Neighbourhood'] = np.where(df2_1['Neighbourhood']=='Not assigned',df2_1['Borough'],df2_1['Neighbourhood'])

In [8]:
df2_1.head()
#df2_1.shape

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df3 = pd.merge(df2_1,df)
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]

In [11]:
df4.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [12]:
mapping_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,long,borough,Neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}-{}'.format(Neighbourhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.8,
    parse_html=False
    ).add_to(mapping_toronto)
    
mapping_toronto  
    
    
    




In [13]:
from sklearn.cluster import KMeans

In [14]:
clustering_toronto = df4.drop(['Postal Code','Borough','Neighbourhood'],1)

In [16]:
clustering_toronto.head()

,Latitude,Longitude
2,43.654260,-79.360636
4,43.662301,-79.389494
9,43.657162,-79.378937
15,43.651494,-79.375418
19,43.676357,-79.293031


In [19]:
km = KMeans(n_clusters=5,random_state=0).fit(clustering_toronto)
km.labels_
df4.insert(0,'Cluster Labels',km.labels_)

In [21]:
df4.head(10)

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,2,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [32]:
k_cluster = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# Initially arrange the number of centroids
# Then for K(No of centroid) of times use this formaul i+x+(i*x)**2 for i=0 to 5
#initialize array which holds "rainbow" of matplotlib from 0 to 1 which equi distance of len(ys)
# use a list where use "colors" of matplotlib
x = np.arange(5)
ys =[i + x + (i*x)**2 for i in range(5)]
color_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in color_array]                    

marker_colors =[]

for lat,long,neighbourhood,cluster in zip(df4['Latitude'],df4['Longitude'],df4['Neighbourhood'],df4['Cluster Labels']):
    label = folium.Popup(' Cluster '+ str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(k_cluster)
k_cluster


